In [22]:
import pulp as pl
import pandas as pd

file_path = 'production-per-rotation-v9.xlsx'

data = pd.read_excel(file_path, sheet_name='Sheet1')

crops = ['Rapeseed', 'Wheat', 'Sunflower', 'Maize', 'Faba bean', 'Chickpea', 'Pea', 'Soybean', 'Barley', 'Oat', 'Triticale', 'Potato']
# Select from below scenarios which one is wanted:
#SCENARIO A- HighProc
# goals = [-186897.0693,  # Rapeseed goal
# -25378859.12,  # Wheat goal    
# -781028.6937,  # Sunflower goal 
# -12924972.07,  # Maize goal  
# 167901.0522,  # Faba bean
# 512931.367,  # Chickpea goal
# 423406.1034,  # Pea goal
# 10960811.41,  # Soybean goal
# -9435892.019,  # Barley goal
# -1708598.412,  # Oat goal      
# -40847.27302,  # Triticale goal 
# -63304.06923] # Potato goal

#SCENARIO B1- MildProc_kcal
# goals = [-186897.0693,  # Rapeseed goal
# -25855676.98,  # Wheat goal    
# -781028.6937,  # Sunflower goal 
# -13003384.41,  # Maize goal  
# 2303169.158,  # Faba bean
# 3139901.024,  # Chickpea goal
# 2186058.06,  # Pea goal
# 6080441.732,  # Soybean goal
# -9435892.019,  # Barley goal
# -1708598.412,  # Oat goal      
# -40847.27302,  # Triticale goal 
# -440807.8035] # Potato goal

# # #SCENARIO B2- MildProc_prot
# goals = [-186897.0693,  # Rapeseed goal
# -25855676.98,  # Wheat goal    
# -781028.6937,  # Sunflower goal 
# -13003384.41,  # Maize goal  
# 2963717.766,  # Faba bean
# 4040424.22,  # Chickpea goal
# 2942497.544,  # Pea goal 
# 3700990.547,  # Soybean goal
# -9435892.019,  # Barley goal
# -1708598.412,  # Oat goal      
# -40847.27302,  # Triticale goal 
# -440807.8035] # Potato goal

#SCENARIO C- MildCap
goals = [-186897.0693,  # Rapeseed goal
-25855676.98,  # Wheat goal    
-781028.6937,  # Sunflower goal 
-13003384.41,  # Maize goal  
3547506.047,  # Faba bean
3547506.047,  # Chickpea goal
3096047.247,  # Pea goal
5501006.884,  # Soybean goal
-9435892.019,  # Barley goal
-1708598.412,  # Oat goal      
-40847.27302,  # Triticale goal 
-440807.8035] # Potato goal


# TOTAL LAND FROM EXCEL
total_arable_land = {
    'Bulgaria': 3416678,
    'Romania': 8801800,
    'Serbia': 2599790,
    'Ukraine': 32717800,
    'Austria': 1345631,
    'Germany': 11812600,
    'UK': 6098618,
    'Ireland': 458881,
    'Italy': 6810645,
    'Belarus': 5632100,
    'Moldova': 1756260,
    'Lithuania': 2195960,
    'Estonia': 661290,
    'Croatia': 855850
}


# Indices for each country (example indices assuming they follow one after the other)
country_indices = {
    'Bulgaria': range(0, 6),
    'Romania': range(6, 17),
    'Serbia': [17],
    'Ukraine': [18],
    'Austria': range(19, 22),
    'Germany': range(22, 31),
    'UK': range(31, 50),
    'Ireland': range(50, 52),
    'Italy': range(52, 59),
    'Belarus': [59],
    'Moldova': [60],
    'Lithuania': [61],
    'Estonia': [62],
    'Croatia': range(63, 65)
}

production_coefficients = data[crops].values

model = pl.LpProblem("Crop_Rotation_Optimization", pl.LpMinimize)
bnd = [(0, None)] * len(data)
x = {}
for i, bounds in enumerate(bnd):
    x[i] = pl.LpVariable(f"Hectares_{i}", lowBound=bounds[0], upBound=bounds[1])


deviations = pl.LpVariable.dicts("Deviation", (range(len(crops))), lowBound=0)

weights = {crop: 100 if crop in ['Faba bean', 'Chickpea', 'Pea', 'Soybean'] else 1 for crop in crops}


model += pl.lpSum(weights[crop] * deviations[j] for j, crop in enumerate(crops))


legume_crops = ['Faba bean', 'Chickpea', 'Pea', 'Soybean']


for j, crop in enumerate(crops):
    total_production = pl.lpSum(data.loc[i, crop] * x[i] for i in range(len(data)) if crop in data.columns)
    model += total_production + deviations[j] >= goals[j], f"Min_Goal_{crop}"
    if crop in legume_crops:
        model += total_production - deviations[j] <= goals[j], f"Max_Goal_{crop}"
       
    
for country, indices in country_indices.items():
    model += pl.lpSum([x[i] for i in indices]) <= total_arable_land[country], f"Arable_Land_{country}"
    
number_of_crops=[4, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 4, 5, 4, 4, 5, 4, 3, 4, 3, 3, 3, 5, 5, 4, 4, 5, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 2, 2, 2, 2, 2, 5, 5, 4, 4, 5, 5, 2, 4]

# data['Non_Zero_Crops'] = (data[crops].astype(bool)).sum(axis=1)
data['Non_Zero_Crops'] = number_of_crops


for country, indices in country_indices.items():
    legume_land = pl.lpSum(
        x[i] * (1.0 / data.loc[i, 'Non_Zero_Crops']) for i in indices if data.loc[i, 'Non_Zero_Crops'] > 0
    )
    model += legume_land <= 0.20 * total_arable_land[country], f"Legume_Land_{country}"



model.solve()


results = {i: pl.value(x[i]) for i in range(len(data))}

results_df = pd.DataFrame(list(results.items()), columns=['Rotation', 'Hectares_Allocated'])
print(results_df)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sophiesaget/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/rv/pplvzh293_zdw65p3n03f51w0000gn/T/df43d05ba2c042188aa6c76e9717522e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/rv/pplvzh293_zdw65p3n03f51w0000gn/T/df43d05ba2c042188aa6c76e9717522e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 49 COLUMNS
At line 528 RHS
At line 573 BOUNDS
At line 574 ENDATA
Problem MODEL has 44 rows, 77 columns and 466 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 32 (-12) rows, 72 (-5) columns and 428 (-38) elements
Perturbing problem by 0.001% of 1201.2814 - largest nonzero change 0.00053251057 ( 0.00019893205%) - largest zero change 0.00051778746
0  Obj 0 Primal inf 11956077 (4)
16  Obj 640.29509
Optimal - objective value 0
After Postsolve, objective 0,

In [23]:
results_df.to_excel("clca-rotations-results-AUG2025_MildCap.xlsx") 

In [24]:
# Example of post-solve check
for j, crop in enumerate(crops):
    if crop in legume_crops:
        achieved = sum(pl.value(x[i]) * data.iloc[i][crop] for i in range(len(data)))
        print(f"Goal for {crop}: {goals[j]}, Achieved: {achieved}")


Goal for Faba bean: 3547506.047, Achieved: 3547505.9853538466
Goal for Chickpea: 3547506.047, Achieved: 3547506.0004166667
Goal for Pea: 3096047.247, Achieved: 3096047.22
Goal for Soybean: 5501006.884, Achieved: 5501006.885027472


In [25]:
# Calculate and print legume land occupation for each country
for country, indices in country_indices.items():
    legume_land_occupied = sum(
        pl.value(x[i]) * (1.0 / data.loc[i, 'Non_Zero_Crops']) for i in indices if data.loc[i, 'Non_Zero_Crops'] > 0
    )
    print(f"Legume land occupied for {country}: {legume_land_occupied:.2f} hectares")
    print(f"20% of total arable land for {country}: {(0.20 * total_arable_land[country]):.2f} hectares")

Legume land occupied for Bulgaria: 228324.02 hectares
20% of total arable land for Bulgaria: 683335.60 hectares
Legume land occupied for Romania: 1760359.99 hectares
20% of total arable land for Romania: 1760360.00 hectares
Legume land occupied for Serbia: 519958.00 hectares
20% of total arable land for Serbia: 519958.00 hectares
Legume land occupied for Ukraine: 0.00 hectares
20% of total arable land for Ukraine: 6543560.00 hectares
Legume land occupied for Austria: 0.00 hectares
20% of total arable land for Austria: 269126.20 hectares
Legume land occupied for Germany: 0.00 hectares
20% of total arable land for Germany: 2362520.00 hectares
Legume land occupied for UK: 1219723.58 hectares
20% of total arable land for UK: 1219723.60 hectares
Legume land occupied for Ireland: 91776.20 hectares
20% of total arable land for Ireland: 91776.20 hectares
Legume land occupied for Italy: 1323090.97 hectares
20% of total arable land for Italy: 1362129.00 hectares
Legume land occupied for Belarus: